# 7.DenseNet
## 7.1 加载包

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.7.0
cuda


## 7.2 定义conv_block卷积块及DenseBlock稠密块

In [2]:
def conv_block(in_channels, out_channels):
    blk = nn.Sequential(nn.BatchNorm2d(in_channels), 
                        nn.ReLU(),
                        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
    # 为什么在最后加上卷积层
    return blk


In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, in_channels, out_channels):
        super(DenseBlock, self).__init__()
        net = []
        for i in range(num_convs):
            in_c = in_channels + i * out_channels  #1-3-5
            net.append(conv_block(in_c, out_channels))
        self.net = nn.ModuleList(net)
        self.out_channels = in_channels + num_convs * out_channels # 计算输出通道数
        
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)  # 在通道维上将输入和输出连结
        return X

### Ex：观察样本经DenseBlock变化

In [4]:
# blk = DenseBlock(2, 3, 10)
# print(blk)
# X = torch.rand(4, 3, 8, 8)
# Y = blk(X)
# Y.shape

## 7.3 定义transition_block过渡层

In [5]:
def transition_block(in_channels, out_channels):
    blk = nn.Sequential(
            nn.BatchNorm2d(in_channels), 
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.AvgPool2d(kernel_size=2, stride=2))
    return blk

### Ex：观察样本经transition_block变化

In [6]:
# blk = transition_block(23, 10)
# blk(Y).shape

## 7.4 定义DenseNet模型

In [7]:
net = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
        #为什么总是用7
        nn.BatchNorm2d(64), 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [9]:
num_channels, growth_rate = 64, 32  # num_channels为当前的通道数
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    DB = DenseBlock(num_convs, num_channels, growth_rate)
    net.add_module("DenseBlosk_%d" % i, DB)
    # 上一个稠密块的输出通道数
    num_channels = DB.out_channels
    # 在稠密块之间加入通道数减半的过渡层
    if i != len(num_convs_in_dense_blocks) - 1:
        net.add_module("transition_block_%d" % i, transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [10]:
net.add_module("BN", nn.BatchNorm2d(num_channels))
net.add_module("relu", nn.ReLU())
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, num_channels, 1, 1)
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(), nn.Linear(num_channels, 10))) 

### Ex：观察样本经DenseNet网络变化

In [11]:
# X = torch.rand((1, 1, 96, 96))
# for name, layer in net.named_children():
#     X = layer(X)
#     print(name, ' output shape:\t', X.shape)

## 7.5 获取数据并训练模型
> 调用数据读取函数d2l.load_data_fashion_mnist、准确率计算函数d2l.evaluate_accuracy及训练函数d2l.train_ch5,如需修改可到d2l的util包中修改

In [18]:
batch_size = 256
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.4530, train acc 0.838, test acc 0.854, time 28.0 sec
epoch 2, loss 0.2711, train acc 0.901, test acc 0.850, time 27.6 sec
epoch 3, loss 0.2321, train acc 0.915, test acc 0.886, time 27.6 sec
epoch 4, loss 0.2093, train acc 0.923, test acc 0.888, time 27.6 sec
epoch 5, loss 0.1928, train acc 0.929, test acc 0.922, time 27.6 sec
